## 一、加载 Qwen1.5-0.5B（远程加载）

In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
import json
from datasets import Dataset
import accelerate
import warnings
warnings.filterwarnings('ignore')

model_name = "Qwen/Qwen1.5-0.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="cpu")
print(f"模型 {model_name} 加载完成")
print(f"模型参数量: {model.num_parameters()/1e6:.2f}M")

模型 Qwen/Qwen1.5-0.5B 加载完成
模型参数量: 463.99M


---
## 二、Prompt 设计问题

### 问题 1：Prompt 结构设计

分别针对**文本摘要**、**问题回答**、**角色扮演**，设计3种不同类型的Prompt。

In [9]:
# 定义三种任务的Prompt模板
prompt_templates = {
    # 任务A：文本摘要（改进：要求明确字数）
    "summarization": """请用50字左右概括以下内容，需包含关键数据：
{input}
精简摘要：""",
    
    # 任务B：问题回答（改进：要求完整回答+结构化输出）
    "qa": """请作为专业健康顾问完整回答以下问题，至少列出5个要点：
问题：{input}
完整回答：""",
    
    # 任务C：角色扮演（改进：明确专业领域）
    "role_playing": """你是一名三甲医院心内科主任医师，请用专业但易懂的方式回答：
患者问：{input}
医生答："""
}
def generate_response(prompt, max_length=300):
    inputs = tokenizer(prompt, 
                      return_tensors="pt",
                      truncation=True,
                      max_length=512)
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_length,  # 增加生成长度
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        num_beams=3,               # 束搜索提高质量
        early_stopping=True,       # 自然停止
        eos_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=2     # 避免重复
    )
    
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return full_response[len(prompt):]  # 返回纯回答内容


#### 问题 2：Prompt 区别分析


In [10]:
test_input = """
最新发表在《美国心脏病学杂志》上的一项长期追踪研究表明，每日保持30分钟的中等强度体育锻炼，如快走、游泳或骑自行车，能够显著降低心血管疾病发病风险约35-40%。这项研究跟踪了超过12,000名年龄在40-75岁之间的参与者，历时15年。

研究负责人、哈佛医学院的约翰·史密斯教授指出，规律运动不仅对心脏健康有益，还能有效缓解焦虑和抑郁症状。数据分析显示，坚持锻炼的参与者心理健康评分平均提高了22%，睡眠质量也有明显改善。

此外，研究还发现，运动带来的健康效益具有累积效应。每周累计150分钟的运动时间，分散在3-5天内完成，就能达到最佳效果。史密斯教授特别强调，对于久坐不动的办公室人群，即使只是每隔一小时站起来活动5分钟，也能带来明显的健康改善。

这项研究的重要意义在于，它证实了适度运动比高强度训练更适合普通人群。研究团队建议，公共卫生政策应该更加重视推广这种简单易行的健康生活方式，而不是过度强调需要专业指导的高强度训练。
"""
print("\n=== 不同Prompt效果测试 ===")
test_question = "如何保持心脏健康？"
test_article = """
最新发表在《美国心脏病学杂志》上的一项长期追踪研究表明，每日保持30分钟的中等强度体育锻炼，如快走、游泳或骑自行车，能够显著降低心血管疾病发病风险约35-40%。这项研究跟踪了超过12,000名年龄在40-75岁之间的参与者，历时15年。

研究负责人、哈佛医学院的约翰·史密斯教授指出，规律运动不仅对心脏健康有益，还能有效缓解焦虑和抑郁症状。数据分析显示，坚持锻炼的参与者心理健康评分平均提高了22%，睡眠质量也有明显改善。

此外，研究还发现，运动带来的健康效益具有累积效应。每周累计150分钟的运动时间，分散在3-5天内完成，就能达到最佳效果。史密斯教授特别强调，对于久坐不动的办公室人群，即使只是每隔一小时站起来活动5分钟，也能带来明显的健康改善。

这项研究的重要意义在于，它证实了适度运动比高强度训练更适合普通人群。研究团队建议，公共卫生政策应该更加重视推广这种简单易行的健康生活方式，而不是过度强调需要专业指导的高强度训练。
"""

print("=== 改进后的Prompt效果测试 ===")
for task_name, template in prompt_templates.items():
    input_content = test_question if task_name != "summarization" else test_article
    prompt = template.format(input=input_content)
    
    print(f"\n【{task_name}】Prompt结构：\n{prompt}")
    
    response = generate_response(prompt)
    
    # 完整性检查（新增功能）
    if not response.endswith(("。", "！", "？")):
        print("⚠️ 检测到回答不完整，尝试补充...")
        continuation = generate_response(prompt + response, max_length=100)
        response += continuation
    
    print(f"\n模型回答：\n{response}\n{'='*50}")


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



=== 不同Prompt效果测试 ===
=== 改进后的Prompt效果测试 ===

【summarization】Prompt结构：
请用50字左右概括以下内容，需包含关键数据：

最新发表在《美国心脏病学杂志》上的一项长期追踪研究表明，每日保持30分钟的中等强度体育锻炼，如快走、游泳或骑自行车，能够显著降低心血管疾病发病风险约35-40%。这项研究跟踪了超过12,000名年龄在40-75岁之间的参与者，历时15年。

研究负责人、哈佛医学院的约翰·史密斯教授指出，规律运动不仅对心脏健康有益，还能有效缓解焦虑和抑郁症状。数据分析显示，坚持锻炼的参与者心理健康评分平均提高了22%，睡眠质量也有明显改善。

此外，研究还发现，运动带来的健康效益具有累积效应。每周累计150分钟的运动时间，分散在3-5天内完成，就能达到最佳效果。史密斯教授特别强调，对于久坐不动的办公室人群，即使只是每隔一小时站起来活动5分钟，也能带来明显的健康改善。

这项研究的重要意义在于，它证实了适度运动比高强度训练更适合普通人群。研究团队建议，公共卫生政策应该更加重视推广这种简单易行的健康生活方式，而不是过度强调需要专业指导的高强度训练。

精简摘要：


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



模型回答：
最新研究证明，每天保持半小时的有氧运动可以降低心脏病和中风的风险。该研究追踪了近1.2万名参与者，并发现每周至少锻炼3次或5次的人群，其心理健康和睡眠表现更好。

【qa】Prompt结构：
请作为专业健康顾问完整回答以下问题，至少列出5个要点：
问题：如何保持心脏健康？
完整回答：


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



模型回答：
 1. 饮食健康：保持均衡的饮食，摄入足够的蛋白质、碳水化合物、脂肪、维生素和矿物质。
2. 锻炼身体：每周至少进行150分钟的中等强度有氧运动，如快走、慢跑、游泳、骑自行车等。
3. 控制体重：过重或肥胖会增加心脏疾病的风险，因此需要控制体重。
4. 戒烟限酒：吸烟和饮酒会损害心脏，应尽量避免。
5. 定期体检：定期进行心电图、血压、血脂、血糖等检查，及时发现和治疗潜在的健康问题。

【role_playing】Prompt结构：
你是一名三甲医院心内科主任医师，请用专业但易懂的方式回答：
患者问：如何保持心脏健康？
医生答：

模型回答：
保持健康的心脏需要良好的生活习惯和饮食习惯。首先，要保证充足的睡眠，每天睡眠时间不少于7小时。其次，饮食要均衡，多吃新鲜蔬菜和水果，少吃高脂肪、高热量的食物，如油炸食品、甜食等。此外，还要戒烟戒酒，避免过度劳累和情绪波动。最后，定期进行体检和心电图检查，及时发现和治疗心脏疾病。




### 任务 2：探索 Prompt 影响

#### 问题 3：最短 Prompt
找到最短仍然能理解的Prompt：

In [15]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen1.5-0.5B", device_map="cpu")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-0.5B")

# 最小化Prompt测试集
minimal_prompts = [
    ("问答", "光速？"),          # 3字符
    ("摘要", "概括：光速"),      # 5字符 
    ("翻译", "英→中：hello"),    # 6字符
    ("代码", "py打印hello"),     # 7字符
    ("极简", "光速")            # 2字符（测试下限）
]

for task, prompt in minimal_prompts:
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        **inputs,
        max_new_tokens=50,
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    print(f"【{task}】Prompt: '{prompt}'\n   → {response[len(prompt):].strip()}\n")

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


【问答】Prompt: '光速？'
   → （　　）
A. 10m/s
B. 约11.2km/s（1m=1×18km）
C. 大约1光年
D. 小于1km/h



Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


【摘要】Prompt: '概括：光速'
   → 是宇宙中最快的速度，是人类目前所能观测到的最快速度。光的速度是每秒299792458米，约为每秒钟300亿公里。在宇宙中的任何地方，光都可以以



Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


【翻译】Prompt: '英→中：hello'
   → →hello
故答案为：Hello．
解析：你好，你好．



Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


【代码】Prompt: 'py打印hello'
   → world
print("hello")
print("")
print("\n")
# 1. 2.3.4.5.6.7.8.9.10.

【极简】Prompt: '光速'
   → 为3×108m/s，光在真空中传播的速度为c=3.0× 1 0 8 m/s，则光从地球到月球需要的时间为（　　）
A. 2.5



#### 问题 4：改变 Prompt 语气
如增加“请详细回答”，对输出有何影响？

In [13]:
tone_variations = [
    {"name": "简洁版", "prompt": "请用1句话简单回答：{input}", "params": {"temperature": 0.3}},
    {"name": "专业版", "prompt": "请用专业术语分点解释：{input}\n1.", "params": {"temperature": 0.5, "num_beams": 3}},
    {"name": "通俗版", "prompt": "用小学生能听懂的话回答：{input}", "params": {"top_k": 40}},
    {"name": "严谨版", "prompt": "{input}（需提供3个科学依据）", "params": {"do_sample": False}},
    {"name": "角色版", "prompt": "你是有30年经验的化学教授，正在给大学生授课。学生问：{input}", "params": {"num_return_sequences": 1}}
]

print("\n=== 修正版语气影响测试 ===")
for tone in tone_variations:
    prompt = tone["prompt"].format(input="水的化学分子式是什么?")
    print(f"\n🔹 {tone['name']} Prompt: '{prompt}'")
    
    # 修正的输入处理
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    )
    
    # 动态生成参数
    base_params = {
        **inputs,
        "max_new_tokens": 150,
        "early_stopping": True,
        "eos_token_id": tokenizer.eos_token_id,
        "pad_token_id": tokenizer.eos_token_id,
        "no_repeat_ngram_size": 2,  # 防止重复
        "num_beams": 1
    }
    params = {**base_params, **tone["params"]}
    
    try:
        outputs = model.generate(**params)
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # 提取生成部分（去除Prompt）
        generated = response[len(prompt):].strip()
        
        # 二次校验（防止截断）
        if not generated or len(generated) < 3:
            raise ValueError("生成内容过短")
            
        print(f"💡 模型回答:\n{generated}")
        
    except Exception as e:
        print(f"❌ 生成失败: {str(e)}")
    
    print("━"*60)


=== 修正版语气影响测试 ===

🔹 简洁版 Prompt: '请用1句话简单回答：水的化学分子式是什么?'
💡 模型回答:
水的分子结构是H2O，由两个氢原子和一个氧原子组成。
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

🔹 专业版 Prompt: '请用专业术语分点解释：水的化学分子式是什么?
1.'
💡 模型回答:
水分子由两个氢原子和一个氧原子组成。
2. 在水分子中，氢和氧的原子个数比为1:2，即1个氢分子和2个氧分子。
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

🔹 通俗版 Prompt: '用小学生能听懂的话回答：水的化学分子式是什么?'
💡 模型回答:
水分子的结构式是怎样的?

水是H2O,分子中H原子和O原子以共价键结合,而O和H之间以非极性键相连,所以水是一种双原子分子,其结构简式为H-O-O-H.
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

🔹 严谨版 Prompt: '水的化学分子式是什么?（需提供3个科学依据）'
💡 模型回答:
水的分子结构是H2O,分子量是18,水分子中氢原子和氧原子的个数比是2:1,所以水是双原子分子,化学式是 H2 O, 每个水原子的质量是 1.80614×10-26kg,每个水离子的质量为 3.2035×  2 6  kg, 一个水气分子的质量 0.001278×22.4  g,一个氢离子质量 9.11×3 4 g  求水在水溶液中电离出的离子的物质的量浓度,
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

🔹 角色版 Prompt: '你是有30年经验的化学教授，正在给大学生授课。学生问：水的化学分子式是什么?'
💡 模型回答:
教授回答：是H2O。你认为教授的回答是（ ）。
A. 错误的，因为水分子中没有氢原子
B. 正确的，在水溶液中，氢离子和氢氧根离子以离子键结合
C. 有误，水是共价化合物
D. 不确定，要看水是什么状态
答案：B
━━━━━━━━━━━━━━━━━━━━━━━━━━━━



## 三、LoRA 微调问题

### LoRA 相关库与配置


In [7]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, PeftModel
from datasets import load_dataset
import matplotlib.pyplot as plt

# 1. 加载Tokenizer和模型
model_name = "Qwen/Qwen1.5-0.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)

### 加载训练数据


In [3]:
from datasets import load_dataset

# 假设你的数据文件为 train.jsonl，格式为每行一个{"input": "...", "output": "..."}
data = load_dataset("json", data_files="train_data.json")["train"]

def preprocess(example):
    prompt = example["input"]
    answer = example["output"]
    full_prompt = prompt + tokenizer.eos_token + answer + tokenizer.eos_token
    tokenized = tokenizer(
        full_prompt, 
        truncation=True, 
        padding="max_length",  # 强制所有样本统一为最大长度
        max_length=512         # 根据模型最大支持长度调整
    )
    tokenized["labels"] = tokenized["input_ids"].copy()  # 确保 labels 与 input_ids 对齐
    return tokenized

tokenized_data = data.map(
    preprocess,
    remove_columns=data.column_names  # 删除原始 input/output 列，避免干扰
)


Map: 100%|██████████| 192/192 [00:00<00:00, 2335.43 examples/s]


In [10]:
# 检查目录权限
print(f"目录 'lora-5epoch' 存在: {os.path.exists('lora-5epoch')}")
try:
    # 尝试创建测试文件
    with open(os.path.join('lora-5epoch', 'test.txt'), 'w') as f:
        f.write('test')
    print("✓ 目录可写入")
    # 清理测试文件
    os.remove(os.path.join('lora-5epoch', 'test.txt'))
except Exception as e:
    print(f"✗ 目录写入失败: {e}")

目录 'lora-5epoch' 存在: False
✗ 目录写入失败: [Errno 2] No such file or directory: 'lora-5epoch\\test.txt'


### LoRA 微调训练代码


In [11]:
def train_lora(epochs, output_dir):
    base_model = AutoModelForCausalLM.from_pretrained(model_name)
    config = LoraConfig(
        r=4,
        lora_alpha=16,
        target_modules=["q_proj", "v_proj"],
        lora_dropout=0.1
    )
    model = get_peft_model(base_model, config)
    data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, 
    mlm=False  # 设置为 False 以适配 Causal LM
)
    args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=epochs,
        per_device_train_batch_size=4,
        save_strategy="no",
        remove_unused_columns=False,
        logging_steps=1,
        learning_rate=2e-4,
        fp16=False,
    )
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=tokenized_data,
        data_collator=data_collator,
    )
    trainer.train()
    model.save_pretrained(output_dir)
    print(f"LoRA 微调完成, 权重保存在 {output_dir}")

def infer(model, prompt):
    device = torch.device("cpu")
    model = model.to(device)
    assert isinstance(prompt, str), "Prompt must be a string!"
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=128,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

def plot_answer_comparison(question, answers, labels, filename):
    plt.figure(figsize=(10, 4))
    plt.title(f"问题：{question}")
    plt.axis("off")
    y0 = 1.0
    for ans, label in zip(answers, labels):
        plt.text(0, y0, f"{label}:\n{ans}", fontsize=12, verticalalignment="top")
        y0 -= 0.35
    plt.savefig(filename, bbox_inches="tight")
    plt.close()
    print(f"已保存对比图：{filename}")
if __name__ == "__main__":
    # -------------------
    # 训练阶段
    # -------------------
    # 训练5轮和10轮的LoRA模型
    if not os.path.exists("lora-5epoch"):
        train_lora(5, "lora-5epoch")
    if not os.path.exists("lora-10epoch"):
        train_lora(10, "lora-10epoch")

    # -------------------
    # 推理与结果分析
    # -------------------
    # 选择一个问题进行分析（可自行替换为实际问题）
    test_question = "请简述LoRA的基本思想。"

    # 1. 原始Qwen模型回答
    original_model = AutoModelForCausalLM.from_pretrained(model_name)
    orig_answer = infer(original_model, test_question)

    # 2. LoRA 微调模型（5轮）
    lora5_model = PeftModel.from_pretrained(
        AutoModelForCausalLM.from_pretrained(model_name),
        "lora-5epoch"
    )
    lora5_answer = infer(lora5_model, test_question)

    # 3. LoRA 微调模型（10轮）
    lora10_model = PeftModel.from_pretrained(
        AutoModelForCausalLM.from_pretrained(model_name),
        "lora-10epoch"
    )
    lora10_answer = infer(lora10_model, test_question)

No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
1,2.048100
2,1.934600
3,2.739500
4,2.155400
5,3.217600
6,2.772700
7,2.117200
8,2.306800
9,2.451700
10,2.144800


e:\anaconda3\envs\pytorch2.3.0\Lib\site-packages\peft\utils\other.py:1110: UserWarning: Unable to fetch remote file due to the following error (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /Qwen/Qwen1.5-0.5B/resolve/main/config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000022F1E7B8F10>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: feff0af3-b0d0-4073-a947-c225475e55a1)') - silently ignoring the lookup for the file config.json in Qwen/Qwen1.5-0.5B.
  warnings.warn(


LoRA 微调完成, 权重保存在 lora-5epoch


### 任务 3：微调效果分析

#### 问题 5：训练前后回答对比
- 用原始模型和5轮训练后的模型，回答同一问题，对比结果。



In [19]:
# 问题5：训练前后对比
# 问题5：训练前后对比 - 直接查看文本输出
test_question = "请简述LoRA的基本思想。"

print("="*50)
print(f"问题: {test_question}")
print("="*50)

print("\n【原始Qwen模型回答】")
print(orig_answer)

print("\n" + "-"*50 + "\n")

print("【LoRA微调后（10轮）回答】")
print(lora5_answer)
print("="*50)



问题: 请简述LoRA的基本思想。

【原始Qwen模型回答】
请简述LoRA的基本思想。LoRA的基本思想是：（1）在LoRA中，每个用户都拥有一个唯一的ID，ID是唯一的，且不可更改；（2）每个用户都拥有一个唯一的IP地址，IP地址是唯一的，且不可更改；（3）每个用户都拥有一个唯一的MAC地址，MAC地址是唯一的，且不可更改；（4）每个用户都拥有一个唯一的IP地址和MAC地址，IP地址和MAC地址是不可更改的；（5）每个用户都拥有一个唯一的ID，ID是唯一的，且不可更改；（6）每个用户都拥有一个唯一的IP地址和

--------------------------------------------------

【LoRA微调后（10轮）回答】
请简述LoRA的基本思想。 LoRA的基本思想是将所有交易都记录在一张票据上，然后将票据与票据进行比较，以确定交易是否合法。这种做法可以确保交易的合法性，并且可以防止欺诈行为。LoRA还允许交易者在票据上添加其他信息，例如交易日期、交易金额、交易地点等。这种做法可以提高交易者的信任度，并且可以防止欺诈行为。LoRA还允许交易者在票据上添加其他信息，例如交易日期、交易金额、交易地点等。这种做法可以提高交易者的信任度，并且可以防止欺诈行为。


In [18]:

# 问题6：5轮和10轮对比 - 直接查看文本输出
test_question = "请简述LoRA的基本思想。"

print("="*50)
print(f"问题: {test_question}")
print("="*50)

print("\n【LoRA微调后（5轮）回答】")
print(lora5_answer)

print("\n" + "-"*50 + "\n")

print("【LoRA微调后（10轮）回答】")
print(lora10_answer)
print("="*50) # 问题6：5轮和10轮对比


问题: 请简述LoRA的基本思想。

【LoRA微调后（5轮）回答】
请简述LoRA的基本思想。 LoRA的基本思想是将所有交易都记录在一张票据上，然后将票据与票据进行比较，以确定交易是否合法。这种做法可以确保交易的合法性，并且可以防止欺诈行为。LoRA还允许交易者在票据上添加其他信息，例如交易日期、交易金额、交易地点等。这种做法可以提高交易者的信任度，并且可以防止欺诈行为。LoRA还允许交易者在票据上添加其他信息，例如交易日期、交易金额、交易地点等。这种做法可以提高交易者的信任度，并且可以防止欺诈行为。

--------------------------------------------------

【LoRA微调后（10轮）回答】
请简述LoRA的基本思想。 LoRA的基本思想是将所有业务流程视为一个整体，将每个业务流程视为一个独立的实体。这个实体可以是任何实体，包括客户、供应商、合作伙伴、员工、政府机构等。LoRA 的核心思想是将业务流程视为一个整体，以实现业务流程的自动化和标准化。通过使用自动化工具和技术，LoRA 可以实现业务流程的自动化和标准化，从而提高效率和降低成本。LoRA 的另一个核心思想是将业务流程视为一个整体，以实现业务流程的标准化和自动化。通过使用自动化工具和技术，LoRA 可以实现业务流程的
